In [3]:
import torch
torch.cuda.empty_cache()

In [1]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import json
import requests

processor_answer = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model_answer = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda")

/home/jovyan/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
image = Image.open(requests.get(url, stream=True).raw)

In [10]:
# Specify the path to your .txt file
file_path = 'questions.txt'  # Replace with your file's path

try:
    # Open the file in read mode
    with open(file_path, 'r') as file:
        # Read the contents of the file
        question = file.read()
        cleaned_questions = question.strip().split('\n')
        # Print the contents of the file
        print(cleaned_questions)

except FileNotFoundError:
    print(f"Error: The file at {file_path} was not found.")
except IOError:
    print("Error: An error occurred while reading the file.")

['What is the color of the sky in the image?', "What is the dog's breed?", 'How is the woman dressed?', 'What is the dog doing?', 'What is the woman holding?', 'What time of day does the image appear to be taken?', 'What is the ocean doing?', "What is the woman's expression?", "What is the dog's expression?", 'What is the color of the sand?', 'What is the woman wearing?', 'What is the dog wearing?', "What is the woman's hair color?", "What is the dog's fur color?", "What is the woman's posture?", "What is the dog's posture?", "What is the woman's clothing style?", "What is the dog's activity?", "What is the woman's mood?"]


In [11]:
answers = [] 
for question in cleaned_questions:
    inputs = processor_answer(image, question , return_tensors="pt").to("cuda")
    
    out = model_answer.generate(**inputs)
    output = processor_answer.decode(out[0], skip_special_tokens=True)
    answers.append(output)
print(answers)

['white', 'lab', 'in plaid', 'sitting', 'dog', 'sunset', 'crashing', 'smiling', 'happy', 'white', 'plaid shirt', 'harness', 'brown', 'tan', 'sitting', 'sitting', 'plaid', 'playing', 'happy']


In [12]:
import json

# Pair the questions with answers
qa_data = [{"question": question, "answer": answer} for question, answer in zip(cleaned_questions, answers)]

# Write the generated data to a JSON file
with open("questions_answers(Qwen+Blip).json", "w") as file:
    json.dump(qa_data, file, indent=4)

print("Questions and answers have been written to 'questions_answers(Qwen+Blip).json'")


Questions and answers have been written to 'questions_answers(Qwen+Blip).json'


In [13]:


# Open and load the JSON file
with open('questions_answers(Qwen+Blip).json', 'r') as file:
    data = json.load(file)

# Print the contents of the JSON file
print(json.dumps(data, indent=4))  # This will format the JSON for better readability


[
    {
        "question": "What is the color of the sky in the image?",
        "answer": "white"
    },
    {
        "question": "What is the dog's breed?",
        "answer": "lab"
    },
    {
        "question": "How is the woman dressed?",
        "answer": "in plaid"
    },
    {
        "question": "What is the dog doing?",
        "answer": "sitting"
    },
    {
        "question": "What is the woman holding?",
        "answer": "dog"
    },
    {
        "question": "What time of day does the image appear to be taken?",
        "answer": "sunset"
    },
    {
        "question": "What is the ocean doing?",
        "answer": "crashing"
    },
    {
        "question": "What is the woman's expression?",
        "answer": "smiling"
    },
    {
        "question": "What is the dog's expression?",
        "answer": "happy"
    },
    {
        "question": "What is the color of the sand?",
        "answer": "white"
    },
    {
        "question": "What is the woman wearing?",
 